In [ ]:
import pandas
import pickle
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.neighbors import kneighbors_graph as knng
from scipy.sparse import load_npz, save_npz
from gensim.models import Doc2Vec
from matplotlib import pyplot as plt
import networkx as nx
from unidecode import unidecode

In [1]:
VECOTORS = 'paragraphs50.data.trainables.syn1neg.npy'
DATA = 'paragraphs50.data'
doc_model = Doc2Vec.load('data/paragraphs50.data')
# with open(f"data/{DATA}",'rb') as f:
#     doc_model = pickle.load(f ) 
# with open(f"data/{VECOTORS}",'rb') as f_arr :
#    doc_array = pickle.load(f_arr)
doc_array = doc_model.docvecs.vectors_docs

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
print(doc_array.shape)

(117977, 50)


In [20]:
import pickle 
with open('data/doc_length.data', 'rb') as f:
    art_lens = pickle.load(f)
def reduce_hi_de(num=300):
    for pageid, size in art_lens:
        if size < num:
            try:
                yield doc_model.docvecs.doctags[str(pageid)].offset
            except:
                continue

In [3]:
def get_all_doc(log=False):
    with open("data/history_depth3.csv", 'r', encoding='utf-8') as file:
        csv_file = pandas.read_csv(file)
    i = 0
    to_list = []
    for name, pageid in zip(csv_file["title"], csv_file["pageid"]):
        try:
            yield doc_model.docvecs.doctags[str(pageid)].offset, name
        except:
            i += 1
            if log:
                to_list.append(str(x))
    print(i)
    if log:
        print(to_list)
labels = np.array([v for k, v in get_all_doc()])   

# CREATE KNN GRAPHS

In [24]:
for i in range(5, 30, 10):
    to_delete = sorted(list(reduce_hi_de(i)))
    new_array = np.delete(doc_array,to_delete , axis=0)
    new_labels = np.delete(labels, to_delete, axis=0)
    np.save(f"test/TEST/labels_{i}", new_labels)
    del new_labels
    del to_delete
    X = normalize(new_array)    
    result = knng(X, n_neighbors=10, n_jobs=4, mode='distance')
    save_npz(f"test/TEST/graph_reduced_{i}", result)
    del result

In [23]:
for i in range(30, 401, 30):
    to_delete = sorted(list(reduce_hi_de(i)))
    new_array = np.delete(doc_array,to_delete , axis=0)
    new_labels = np.delete(labels, to_delete, axis=0)
    np.save(f"test/labels_{i}", new_labels)
    del new_labels
    del to_delete
    X = normalize(new_array)    
    result = knng(X, n_neighbors=10, n_jobs=4, mode='distance')
    save_npz(f"test/graph_reduced_{i}", result)
    del result

15993


# DIFFRERENT METHODS OF SAVING

In [5]:
def save_as_gexf(name=300):
    result = load_npz(f"test/graph_reduced_{name}.npz")
    G = nx.from_scipy_sparse_matrix(result, create_using=nx.DiGraph())
    del result
    labels = {k: v for k, v in enumerate(np.load(f"test/labels_{name}.npy"))}
    labels_rev = {v: k for k, v in labels.items()}
    nx.relabel_nodes(G, labels, copy=False)
    nx.set_node_attributes(G, labels_rev,'id')
    nx.write_gexf(G, f"graphs/graph_reduced{i}.gexf")

In [3]:
def get_to_csv(graph, labels):
    for key, value in sorted(labels.items(), key=lambda x: x[0]):
        value = unidecode(value)
        try:
            int(value)
            yield key, int(graph.degree(key)), "YEAR " + value
        except:
            yield key, int(graph.degree(key)), value.replace('_', ' ')

def save_as_csv(graph, labels, i, weight = ['weight']):
    with open( f'{i}.graph', 'w', encoding='utf-8') as f:
        for line in nx.generate_edgelist(graph, data=False):
            f.write(line+'\n')
    with open(f'{i}.desc', 'w', encoding='utf-8') as f:
        text = 'degree, title\nreal, string\n'
        f.write(text)
        for a in get_to_csv(graph, labels):
            f.write(" ".join(map(str, a)) + "\n")

In [4]:
def save_to_ivga(name=300):
    result = load_npz(f"test/graph_reduced_{name}.npz")
    G = nx.from_scipy_sparse_matrix(result, create_using=nx.DiGraph())
    del result
    labels = {k: v for k, v in enumerate(np.load(f"test/labels_{name}.npy"))}
    save_as_csv(G, labels, name)

for i in range(30, 401, 30):
    save_to_ivga(name=i)
for i in range(5, 30, 10):
    save_to_ivga(name=i)

In [20]:
def print_edges(name=None):
    if not name:
        name = "Adolf_Hitler"
    print("In edges:")
    print("\n".join(map(lambda t: f"\t{t[0]} -> {t[1]}", G.in_edges(name))))
    print("Out edges:")
    print("\n".join(map(lambda t: f"\t{t[0]} -> {t[1]}", G.out_edges(name))))
print_edges()

In edges:
	Albert_Speer -> Adolf_Hitler
	Adolf_Hitler's_rise_to_power -> Adolf_Hitler
	Religious_views_of_Adolf_Hitler -> Adolf_Hitler
	Hitler_family -> Adolf_Hitler
	Wehrmacht -> Adolf_Hitler
Out edges:
	Adolf_Hitler -> Albert_Speer
	Adolf_Hitler -> Wannsee_Conference
	Adolf_Hitler -> Night_of_the_Long_Knives
	Adolf_Hitler -> Madagascar_Plan
	Adolf_Hitler -> Frederick_Taylor_(historian)
	Adolf_Hitler -> Adolf_Hitler's_rise_to_power
	Adolf_Hitler -> Religious_views_of_Adolf_Hitler
	Adolf_Hitler -> Reich_Chancellery_meeting_of_12_December_1941
	Adolf_Hitler -> Wehrmacht
	Adolf_Hitler -> Lieberose_forced_labor_camp


In [21]:
print_edges("World_War_II")

In edges:
	Aftermath_of_World_War_I -> World_War_II
	Cold_War -> World_War_II
	Origins_of_the_Cold_War -> World_War_II
	Mobilization -> World_War_II
	Causes_of_World_War_II -> World_War_II
	Middle_East_Theatre_of_World_War_II -> World_War_II
	Mediterranean_and_Middle_East_theatre_of_World_War_II -> World_War_II
	Japan_during_World_War_I -> World_War_II
	World_War_I -> World_War_II
	Aftermath_of_World_War_II -> World_War_II
Out edges:
	World_War_II -> Potsdam_Conference
	World_War_II -> Potsdam_Agreement
	World_War_II -> 196
	World_War_II -> Tehran_Conference
	World_War_II -> Cairo_Conference
	World_War_II -> Events_preceding_World_War_II_in_Europe
	World_War_II -> Middle_East_Theatre_of_World_War_II
	World_War_II -> Mediterranean_and_Middle_East_theatre_of_World_War_II
	World_War_II -> Japan_during_World_War_I
	World_War_II -> 1126_in_art


In [36]:
def n_max_degree(n=10):
    print(f"{n} highest degree:")
    print(", ".join(map(lambda t: t[1], sorted([(d,n) for n, d in G.degree()], reverse=True, key=lambda t: t[0])[:n])))

In [38]:
n_max_degree(150)

150 highest degree:
426, AD_32, 264, 112, Raczkiewicz, 140, AD_29, 99_BC, AD_61, 125, 144, 402_BC, 318, AD_37, AD_41, 2024, AD_31, 817, Mohammed_al-Duayf, Ten'ō, AD_76, 451, 787, AD_20, 373_BC, Suzuko_no_Koi, Rizvan, Bogić, Emergent_materialism, Pejo, 12_BC, Declaration_of_Indulgence, AD_19, December_7, AD_11, 537_BC, Ryujoseph, 278, Chilean_presidential_election,_1856, List_of_peers_1320–1329, Emigrant_Gap, 233, Plasma_processing, List_of_years_in_Greece, 349, 520_BC, 141, Bezruč, 517, 93_BC, United_Nations_Security_Council_Resolution_584, Fidan, 781, Pretoria_Accord, Empirical_knowledge, Sagar,_Madhya_Pradesh, Kadelburg, 816, 772, Hisham_ibn_al-Kalbi, Chang-woo, February_7, Japan_Coast_Guard_Museum, United_Nations_Security_Council_Resolution_735, 304, The_Love_of_Richard_Nixon, Šolontu, Cisapride, 136_BC, Self-evidence, Long_knives, Tuo, 1380_in_art, 754, Issaka, Jess, Web_browser, 2066, 1527_in_music, Hatogaya_Domain, Noubar, February_1, 401_BC, Pulseless_electrical_activity, Svetoz